In [1]:
import csv
import json
import os
from collections import defaultdict

In [2]:
folder = 'results/'
baselines = ['TOKEN', 'FVN-', 'FVN']
files = ['simple_seq2seq_personage_delex_VAE', 'simple_VQVAE_personage_LSTM_delex_beam10', 'controlled_VQVAE_personage_LSTM_delex_beam10']

In [13]:
def read_result_data(fn):
    file = open(fn, 'r')
    meta = []
    while True:
        line = file.readline()
        if 'START_CSV_SECTION' in line:
            break
    meta.append(line)
    reader = csv.DictReader(file)
    data = [_ for _ in reader]
    file.close()
    return data

def pack_dial(data):
    dials = {}
    for turn in data:
        dial_id = int(turn['id'])
        if dial_id not in dials:
            dials[dial_id] = []
        dials[dial_id].append(turn)
    return dials

def dump(dats):
        for k, v in data.items():
            dump_file.write('{}\t{}\n'.format(k, v))


In [25]:
def cleantext(t):
    s_tokens = ['<go> ', ' EOS']
    for s in s_tokens:
        t = t.replace(s, '')
    return t

output = defaultdict(dict)
for baseline, file in zip(baselines, files):
    data = read_result_data(folder+file+'.csv')
    for d in data:
        _id = int(d['id'])
        if  _id not in output:
            output[_id]['id']=d['id']
            output[_id]['slot_value']=d['slot_value']
            output[_id]['personality']=d['personality']
            output[_id]['ground_truth']=cleantext(d['delex_text'])
        
        output[_id][baseline]=cleantext(d['pred_delex_text'])

In [32]:
result_file = open('result_4_humanevaluation.csv', 'w')
writer = csv.DictWriter(result_file, fieldnames=['id', 'slot_value', 'personality', 'ground_truth'] + baselines)
writer.writeheader()
for i, result in sorted(output.items()):
    writer.writerow(result)
result_file.close()   

0 {'id': '0', 'slot_value': '{"name": "nameVariable", "eatType": "restaurant", "food": "fast food", "area": "city centre", "familyFriendly": "no", "near": "nearVariable"}', 'personality': 'agreeable', 'ground_truth': "let 's see , nameVariable ... yeah , right , nameVariable is a eatTypeVariable , it is a fast food eatTypeVariable in city centre , also nameVariable is n't kid friendly , and it 's kind of near nearVariable okay ?", 'TOKEN': "let 's see what ? can find on nameVariable . a foodVariable restaurant in city , , it it , it it is it it it it , , , , , , , , , , , ?", 'FVN-': "let 's see , nameVariable ... right , it is a foodVariable restaurant near nearVariable and rather moderately priced , also it is a eatTypeVariable , you know , you see ?", 'FVN': "let 's see , nameVariable ... right , it is a coffee shop , also it is a foodVariable restaurant near nearVariable in areaVariable , and it is n't somewhat kid friendly ,"}
1 {'id': '1', 'slot_value': '{"name": "nameVariable", 

1268 {'id': '1268', 'slot_value': '{"name": "nameVariable", "eatType": "pub", "food": "French", "priceRange": "20-25", "customerRating": "high", "area": "city centre", "familyFriendly": "yes", "near": "nearVariable"}', 'personality': 'unconscientiousness', 'ground_truth': "oh god oh gosh i do n't know . nameVariable has a price range of £20-25 , it is a foodVariable place near nearVariable , it is family friendly in city centre and it has a customerRatingVariable rating . it is a eatTypeVariable !", 'TOKEN': 'oh god i am i am . know . nameVariable is a coffee shop , a , rating , , , , , , , , , , , , , , , , , , it it is a .', 'FVN-': "oh god yeah , i do n't know . nameVariable is a foodVariable place near nearVariable with a customerRatingVariable rating ,", 'FVN': "oh god i do n't know . nameVariable is a coffee shop , also it is darn kid friendly , also it has like , a customerRatingVariable rating , also it is a foodVariable place in areaVariable , and it is moderately priced , als